[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/master/bayesian_optimization.ipynb)

In [1]:
try:
    # Check if notebook is running in Google Colab
    import google.colab
    # Get additional files from Github
    !wget https://raw.githubusercontent.com/krasserm/bayesian-machine-learning/master/bayesian_optimization_util.py
    # Install additional dependencies
    !pip install scikit-optimize==0.5.2
    !pip install GPy==1.9.8
    !pip install GPyOpt==1.2.1
    !pip install xgboost==0.90
except:
    pass

--2024-03-22 19:06:21--  https://raw.githubusercontent.com/krasserm/bayesian-machine-learning/master/bayesian_optimization_util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1536 (1.5K) [text/plain]
Saving to: ‘bayesian_optimization_util.py’

bayesian_optimizati 100%[===================>]   1.50K  --.-KB/s    in 0s      

2024-03-22 19:06:21 (18.2 MB/s) - ‘bayesian_optimization_util.py’ saved [1536/1536]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.5/989.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.0 MB/s eta 0:00:00
  Created wheel for GPy: filename=GPy-1.9.8-cp310-cp310-linux_x86_64

In [2]:
import numpy as np

%matplotlib inline

bounds = np.array([[1,4],[15,35]])

noise = 0

def f(X, noise=noise):
    return [-np.sin(3*X[0]) - X[1]**2 + 0.7*X[0]]



The following plot shows the noise-free objective function, the amount of noise by plotting a large number of samples and the two initial samples.

In [3]:
import matplotlib.pyplot as plt

# Dense grid of points within bounds
#X = [np.arange(bounds[:, 0], bounds[:, 1], 0.1).reshape(-1, 1), np.arange(bounds[:, 0], bounds[:, 1], 0.1).reshape(-1, 1)]

# Noise-free objective function values at X
#Y = f(X,0)

# Plot optimization objective with noise level
#plt.plot(X, Y, 'y--', lw=2, label='Noise-free objective')
#plt.plot(X, f(X,0), 'bx', lw=1, alpha=0.1, label='Noisy samples')
#plt.plot(X_init, Y_init, 'kx', mew=3, label='Initial samples')

#plt.legend();

Goal is to find the global optimum on the left in a small number of steps. The next step is to implement the acquisition function defined in Equation (2) as `expected_improvement` function.

In [4]:
from scipy.stats import norm

def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.

    Args:
        X: Points at which EI shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.

    Returns:
        Expected improvements at points X.
    '''
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)

    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    # See also section 2.4 in [...]
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

We also need a function that proposes the next sampling point by computing the location of the acquisition function maximum. Optimization is restarted `n_restarts` times to avoid local optima.

In [5]:
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.

    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)

    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun
            #min_val = res.fun[0]
            min_x = res.x

    return min_x.reshape(-1, 1)

Now we have all components needed to run Bayesian optimization with the [algorithm](#Optimization-algorithm) outlined above. The Gaussian process in the following example is configured with a [Matérn kernel](http://scikit-learn.org/stable/modules/gaussian_process.html#matern-kernel) which is a generalization of the squared exponential kernel or RBF kernel. The known noise level is configured with the `alpha` parameter.

Bayesian optimization runs for 10 iterations. In each iteration, a row with two plots is produced. The left plot shows the noise-free objective function, the surrogate function which is the GP posterior predictive mean, the 95% confidence interval of the mean and the noisy samples obtained from the objective function so far. The right plot shows the acquisition function. The vertical dashed line in both plots shows the proposed sampling point for the next iteration which corresponds to the maximum of the acquisition function.

In [7]:
import math
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
def closest(pt, others):
    return min(others, key = lambda i: distance(pt, i))

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

In [9]:
!conda install matplotlib=2.2.3


/bin/bash: line 1: conda: command not found


In [10]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from bayesian_optimization_util import plot_approximation, plot_acquisition
from sklearn.gaussian_process.kernels import RBF,ConstantKernel as C,RationalQuadratic as RQ,WhiteKernel,ExpSineSquared as Exp,DotProduct as Lin
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [11]:
def predict(gpr, prediction, i, X_pool, Y_pool, X_next, X_next_availble):

    if i == 0:
        prediction = []
        # Initialize samples
        global X_sample
        X_sample = X_init
        global Y_sample
        Y_sample = Y_init

        # Number of iterations
        n_iter = 10

        plt.figure(figsize=(12, n_iter * 3))
        plt.subplots_adjust(hspace=0.4)



    # Update Gaussian process with existing samples
    gpr.fit(X_sample, Y_sample)


    X_next = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds)
    X_next_availble = closest(X_next, X_pool)


    # Obtain next closest sample from the avalible sample pool
    #Y_next_availble = f(X_next_availble, noise)
    t1 = np.argwhere(X_pool[:,0] == X_next_availble[0])
    t2 = np.argwhere(X_pool[:,1] == X_next_availble[1])
    index = np.intersect1d(t1,t2)
    Y_next_availble = Y_pool[index]




    # Add sample to previous samples
    X_sample = np.vstack((X_sample, X_next_availble))
    Y_sample = np.vstack((Y_sample, Y_next_availble))


    # Reshape X_next
    X_next = np.reshape(X_next, (1, 2))


    print("the next suggestion is {}, predicted to be {}".format(X_next, gpr.predict(X_next)))
    print(" ")
    print("Next availble sample: {}".format(X_next_availble))
    prediction.append(float(gpr.predict(X_next)))
    print(" ")
    print("Predition history: ")
    print(prediction)
    print(" ")
    print("Samples used: ")
    print(X_sample)
    print(" ")
    print("Samples' value used: ")
    print(Y_sample)
    print(" ")





    # Plot 3D pridictions
    fig = plt.figure(figsize=(10,6))
    ax1 = fig.add_subplot(111, projection='3d')

    x1 = np.arange(0.9,4.2,0.1)
    x2 = np.arange(14,36,0.5)
    X1,X2 = np.meshgrid(x1,x2)

    test1 = np.reshape(X1,(1452,1))
    test2 = np.reshape(X2,(1452,1))
    X12 = np.concatenate((test1, test2), axis=1)

    Y = gpr.predict(X12)
    Y = np.reshape(Y,(44, 33))

    mycmap = plt.get_cmap('gist_earth')
    ax1.set_title('Iteration {}'.format(i))
    surf1 = ax1.plot_surface(X1, X2, Y, cmap=mycmap)
    ax1.scatter(X_sample[:-1,0], X_sample[:-1,1], Y_sample[:-1], color = 'black')
    ax1.scatter(X_next[0][0], X_next[0][1], Y_sample[-1], color='orange')
    ax1.scatter(X_next_availble[0], X_next_availble[1], Y_sample[-1], color='red')
    fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
    plt.show()

    plot = plt.pcolormesh(X1, X2, Y, cmap='RdBu', shading='auto')

    # Plot 2D pridictions
    levels = np.arange(97,100,0.01)
    cset = plt.contour(X1, X2, Y, cmap='gray')
    plt.clabel(cset, inline=True)
    plt.plot(X_sample[:,0], X_sample[:,1], 'o', color='black')
    plt.plot(X_next[0][0], X_next[0][1], 'o', color='orange');
    plt.plot(X_next_availble[0], X_next_availble[1], 'o', color='red')
    plt.colorbar(plot)


    # Remove used sample
    index_to_remove = np.where(np.all(X_pool == X_next_availble,axis=1))
    X_pool = np.delete(X_pool, index_to_remove, 0)
    Y_pool = np.delete(Y_pool, index_to_remove, 0)



    i +=1

    return i, prediction, X_pool, Y_pool, X_next, X_next_availble

In [12]:
# initialize
i = 0
prediction = []
X_next = []
X_next_availble = []

kernel = 1 * RBF([1,6])
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0)

#X_all = np.array([[-1.33,-0.8], [-1.33,-0.1], [-1.33,0.59],
#
#                 [-0.34,-1.5], [-0.34,-0.8], [-0.34,-0.1], [-0.34,0.59], [-0.34,1.29],
#
#                 [0.6475,-1.5], [0.6475,-0.8], [0.6475,-0.1], [0.6475,0.59], [0.6475,1.29],
#
#                 [1.6358,-0.8], [1.6358,-0.1], [1.6358,0.59],])
#Y_all = np.array([[-1.166], [-1.608], [-2.444],
 #
 #                [0.479], [0.789], [0.888], [0.783], [0.838],
 #
 #                [-0.375], [-0.097], [0.3615], [0.622], [0.5783],
 #               [-0.0658], [0.2376], [0.03325]])


X_all = np.array([[1,15], [1,20], [1,25], [1,30],[1,35],
                  [1.273,27.5],[1.486,20.4],[1.486,34.6],
                 [2,15],[2,17.5], [2,20], [2,25],[2,27.5], [2,30], [2,35],
                  [2.51,20.4],[2.51,34.6],[2.73,27.5],
                  [3,15], [3,20], [3,25], [3,30], [3,35],
                  [4,15],[4,20], [4,25], [4,30],[4,35]])
Y_all = np.array([[98.08],[98.2565], [97.9], [97.225],[97.03],
                  [99.01],[99.76],[99.32],
                  [99.585],[99.66], [99.835], [99.915],[99.88], [99.83], [99.875],
                  [99.76],[99.88],[99.82],
                [98.895], [99.12], [99.49], [99.7], [99.665],
                 [98.385],[99.145], [99.39], [99.225],[99.35]])


X_pool = X_all
Y_pool = Y_all

#X_init = np.array([[1,15], [1,35], [4,15], [4,35]])
X_init = X_all
#Y_init = np.array([[98.08], [97.03], [98.385], [99.35]])
Y_init = Y_all

#X_init = np.array([[-1.33,-1.5], [-1.33,1.29], [1.6358,-1.5], [1.6358,1.29]])

#Y_init = np.array([[-1.3851], [-2.68576], [-1], [0.188]])


In [ ]:
# Predict
i, prediction, X_pool, Y_pool, X_next, X_next_availble = predict(gpr, prediction, i, X_pool, Y_pool, X_next, X_next_availble)